In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import glob
import os
import warnings

import astropy.units as u
from astropy.modeling import models
from astroquery.nist import Nist # atomic lines
# from astroquery.linelists.cdms import CDMS # molecular lines?

from muler.igrins import IGRINSSpectrum, IGRINSSpectrumList

from tqdm import tqdm

from gollum.phoenix import PHOENIXSpectrum, PHOENIXGrid
from muler.hpf import HPFSpectrumList
from specutils import SpectralRegion
from specutils.manipulation import extract_region
from specutils.fitting import find_lines_derivative, fit_continuum

# %config InlineBackend.figure_format='retina'

from astropy.io import fits

# Plotting Parameters
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] =18

plt.rcParams['legend.fontsize'] = 16
plt.rcParams['figure.titlesize'] = 20

plt.rcParams['axes.labelweight']='bold'
plt.rcParams['axes.linewidth'] = 3

plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.minor.visible'] = True
plt.rcParams['xtick.minor.size'] = 5

plt.rcParams['ytick.major.size'] = 10
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['ytick.minor.size'] = 5

plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.direction'] = 'in'

# %matplotlib --list

ModuleNotFoundError: No module named 'gollum.phoenix'

In [ ]:
# Size of 1 spectral resolution element
# IGRINS Spectral Resolution
spec_res = 0.00001

# Reduced and order-merged data filepath 
# Laptop Path
data_path = "C:\\Users\\Savio\\Documents\\GitHub\\IGRINS-Spectra\\IGRINS_Merged"

# File path for figures to live in
fig_path = "C:\\Users\\Savio\\Documents\\GitHub\\IGRINS-SpectraIGRINS_figs\\standards_spectra"

# Create the folder if it doesn't exist
if not os.path.exists(fig_path):
    os.makedirs(fig_path)

# Nicole's merged K-band spectra of some Taurus Standards
# merged_standard_files = glob.glob(data_path + "/merged_standards/m*.fits")
standard_table = pd.read_csv('C:\\Users\\Savio\\Documents\\IGRINS-Spectra\\standard_table_v3.txt', index_col=0)  # csv of standards with file and Spectral Type, c/v TBA
# just a pick a sequence of standards to look at: ["LkCa1","HBC427","Hubble4","Anon1","LkCa5","MHO8"])]
# Symposium sequence: ["LkCa19","HBC427","Hubble4","Anon1","HBC359","LkCa21","LkCa1","MHO8"]
standard_table = standard_table[standard_table['Name'].isin(["LkCa19","HBC427","Hubble4","Anon1","HBC359","LkCa21","LkCa1","MHO8"])].reset_index(drop=True)

proto_table = pd.read_csv('C:\\Users\\Savio\\Documents\\IGRINS-Spectra\\protostar_table.txt', index_col=0)

standards_path = standard_table['File']
standard_list = standard_table['File'].values

proto_path = proto_table['File']
proto_list = proto_table['File'].values

In [ ]:
from bokeh.io import output_notebook

# PHOENIX MODELS

In [ ]:
phoenix_mod_path = "C:\\Users\\Savio\\Documents\\phoenix_models\\phoenix.astro.physik.uni-goettingen.de\\HiResFITS\\"

In [ ]:
standard_table

In [ ]:
test_spec = IGRINSSpectrumList.read(standard_table['File'][0])[14].trim_edges(limits=(100,1948)).normalize().remove_nans()

In [ ]:
grid = PHOENIXGrid(teff_range=(2500, 7000), logg_range=(0, 5), Z_range=(0,0),
                   wl_lo=test_spec.wavelength.value.min(), wl_hi=test_spec.wavelength.value.max(),
                   path=phoenix_mod_path)

In [ ]:
grid.show_dashboard(test_spec, notebook_url='localhost:8888')

In [ ]:
# for i in range(len(proto_table)):
#     try:
#         # Load and process each spectrum
#         test_spec = IGRINSSpectrumList.read(proto_table['File'][i]).trim_overlap().remove_nans().stitch()

#         # Define conditions for masking (example: keep only positive and non-NaN flux values)
#         flux_condition = (test_spec.flux.value > 0) & (~np.isnan(test_spec.flux.value))
#         wavelength_condition = (test_spec.spectral_axis.value > 20800) & (test_spec.spectral_axis.value < 24000)
        
#         # Combine the conditions into a single mask
#         combined_mask = flux_condition & wavelength_condition
        
#         # Apply the mask if it’s not empty
#         masked_spectrum = test_spec.apply_boolean_mask(combined_mask).normalize()

#     except AssertionError as e:
#         # Skip the spectrum if the mask leaves no pixels
#         print(f"Skipping spectrum at index {i} due to masking error: {e}")
#         continue  # Move to the next iteration

#     masked_spectrum.plot()

In [ ]:
template = PHOENIXSpectrum(teff=3000, logg=5, Z=0,wl_lo=np.min(test_spec.wavelength.value),wl_hi=np.max(test_spec.wavelength.value), download=True).normalize().resample(test_spec)

In [ ]:
res = test_spec.flux-template.flux

fig, (ax1,ax2) = plt.subplots(2,1,figsize=(15,5), gridspec_kw={'height_ratios': [4, 1]}, sharex=True)

ax1.plot(test_spec.wavelength, test_spec.flux, label='Data')
ax1.plot(template.wavelength, template.flux, label='PHOENIX Model')
ax1.set_ylabel('Flux')

ax2.scatter(template.wavelength, res,color='r',s=2)
ax2.set_ylim(-0.25,0.25)
ax2.set_xlabel('Wavelength')
ax2.set_ylabel('res')

ax1.legend()
plt.show()

In [ ]:
vsinis = np.linspace(1, 150, 20)
rvs = np.linspace(-100, 100, 20)
search_vsini, search_rv = np.meshgrid(vsinis, rvs, indexing='ij')

In [ ]:
@np.vectorize
def rss(vsini, rv):
    model = template.rotationally_broaden(vsini).rv_shift(rv).instrumental_broaden(45000).resample(test_spec).normalize()
    return np.nansum((test_spec.flux - model.flux)**2)

loss = rss(search_vsini, search_rv)

In [ ]:
best_i, best_j = np.unravel_index(np.argmin(loss), (len(vsinis), len(rvs)))
best_vsini, best_rv = vsinis[best_i], rvs[best_j]
best_vsini, best_rv

In [ ]:
plt.imshow(loss, extent=[rvs.min(), rvs.max(), vsinis.min(), vsinis.max()], aspect='auto', origin='lower', interpolation='gaussian')
plt.scatter(best_rv, best_vsini,  marker='*', c='w', ec='k', s=200)

plt.colorbar(label='RSS')
plt.xlabel(r'$v_r \quad [\mathrm{km}/\mathrm{s}]$')
plt.ylabel(r'$v\sin(i) \quad [\mathrm{km}/\mathrm{s}]$')

plt.show()

In [ ]:
min_wav_idx = np.nanargmin(test_spec.wavelength.value)
max_wav_idx = np.nanargmax(test_spec.wavelength.value)

In [ ]:
best_spec_full = template.rotationally_broaden(best_vsini).rv_shift(best_rv).instrumental_broaden(resolving_power=45000)
best_spec = best_spec_full[min_wav_idx:max_wav_idx].normalize()

In [ ]:
best_spec

In [ ]:
ax = (test_spec/test_spec.flux.max()).plot(marker='.',markersize=1, linestyle='--', lw=1 , color='k', alpha=0.5, label='test_spec')

(best_spec/np.nanmax(best_spec.flux)).plot(ax=ax,label='best_model', color='r', zorder=20)

ax.set_ylim(0.5,1.25)
ax.set_title(rf'$v\sin(i)$ = {best_vsini:0.0f} km/s, $v_r$ = {best_rv:0.1f} km/s')

plt.legend()
plt.show()

In [ ]:
grid[0]